In [1]:
import pandas as pd
import numpy as np
import ast
import os

print("Loading TMDB 5000 movie datasets...")

# Load the dataset files from the data/dataset directory
try:
    movies = pd.read_csv('data/dataset/tmdb_5000_movies.csv')
    credits = pd.read_csv('data/dataset/tmdb_5000_credits.csv')
    
    print(f"Movies shape: {movies.shape}")
    print(f"Credits shape: {credits.shape}")
    
    print("\nMovies columns:")
    print(movies.columns.tolist())
    
    print("\nCredits columns:")
    print(credits.columns.tolist())
    
    print("\nDatasets loaded successfully!")
    
except FileNotFoundError as e:
    print(f"File not found: {e}")
    print("Please make sure the dataset files are in the data/dataset directory")

Loading TMDB 5000 movie datasets...
Movies shape: (4803, 20)
Credits shape: (4803, 4)

Movies columns:
['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']

Credits columns:
['movie_id', 'title', 'cast', 'crew']

Datasets loaded successfully!


In [2]:
print("=== DATASET EXPLORATION ===")

# Explore movies dataset
print("MOVIES DATASET:")
print(f"Shape: {movies.shape}")
print(f"Columns: {list(movies.columns)}")
print(f"\nFirst few rows:")
print(movies.head())
print(f"\nData types:")
print(movies.dtypes)
print(f"\nMissing values:")
print(movies.isnull().sum())

print("\n" + "="*50)

# Explore credits dataset
print("CREDITS DATASET:")
print(f"Shape: {credits.shape}")
print(f"Columns: {list(credits.columns)}")
print(f"\nFirst few rows:")
print(credits.head())
print(f"\nMissing values:")
print(credits.isnull().sum())

# Check unique values for key columns
print(f"\nUnique languages: {movies['original_language'].nunique()}")
print(f"Vote average range: {movies['vote_average'].min():.1f} - {movies['vote_average'].max():.1f}")
print(f"Vote count range: {movies['vote_count'].min()} - {movies['vote_count'].max()}")

=== DATASET EXPLORATION ===
MOVIES DATASET:
Shape: (4803, 20)
Columns: ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']

First few rows:
      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  2

In [3]:
print("=== DATA CLEANING STEP 1: Data Types & Basic Cleaning ===")

# Convert release_date to datetime and extract year
print("Converting date columns...")
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies['release_year'] = movies['release_date'].dt.year

# Convert numeric columns to proper types
print("Converting numeric columns...")
numeric_columns = ['budget', 'revenue', 'popularity', 'vote_average', 'vote_count', 'runtime']
for col in numeric_columns:
    movies[col] = pd.to_numeric(movies[col], errors='coerce')

print("Data type conversions completed!")
print(f"\nUpdated data types for key columns:")
print(movies[['budget', 'revenue', 'popularity', 'vote_average', 'vote_count', 'runtime', 'release_year']].dtypes)

# Show some basic statistics
print(f"\nBasic statistics:")
print(movies[['budget', 'revenue', 'vote_average', 'vote_count', 'release_year']].describe())

=== DATA CLEANING STEP 1: Data Types & Basic Cleaning ===
Converting date columns...
Converting numeric columns...
Data type conversions completed!

Updated data types for key columns:
budget            int64
revenue           int64
popularity      float64
vote_average    float64
vote_count        int64
runtime         float64
release_year    float64
dtype: object

Basic statistics:
             budget       revenue  vote_average    vote_count  release_year
count  4.803000e+03  4.803000e+03   4803.000000   4803.000000   4802.000000
mean   2.904504e+07  8.226064e+07      6.092172    690.217989   2002.468763
std    4.072239e+07  1.628571e+08      1.194612   1234.585891     12.414354
min    0.000000e+00  0.000000e+00      0.000000      0.000000   1916.000000
25%    7.900000e+05  0.000000e+00      5.600000     54.000000   1999.000000
50%    1.500000e+07  1.917000e+07      6.200000    235.000000   2005.000000
75%    4.000000e+07  9.291719e+07      6.800000    737.000000   2011.000000
max   

In [4]:
print("=== DATA CLEANING STEP 2: Remove Duplicates & Filter Quality Data ===")

# Remove duplicate movies based on title
print(f"Original dataset size: {len(movies)} movies")
movies_clean = movies.drop_duplicates(subset=['title'], keep='first').copy()
print(f"After removing duplicates: {len(movies_clean)} movies")

# Apply quality filters for recommender system
print("\nApplying quality filters...")
quality_filter = (
    (movies_clean['vote_count'] >= 100) &      # At least 100 votes for reliability
    (movies_clean['vote_average'] > 0) &       # Valid ratings
    (movies_clean['overview'].notna()) &       # Must have overview
    (movies_clean['overview'] != '') &         # Overview not empty
    (movies_clean['release_year'] >= 1980) &   # Movies from 1980 onwards
    (movies_clean['release_year'] <= 2024) &   # Not future movies
    (movies_clean['runtime'] > 0)             # Valid runtime
)

print(f"Before quality filter: {len(movies_clean)} movies")
movies_clean = movies_clean[quality_filter]
print(f"After quality filter: {len(movies_clean)} movies")

print(f"\nFinal dataset characteristics:")
print(f"Year range: {movies_clean['release_year'].min()} to {movies_clean['release_year'].max()}")
print(f"Vote count range: {movies_clean['vote_count'].min()} to {movies_clean['vote_count'].max()}")
print(f"Vote average range: {movies_clean['vote_average'].min():.1f} to {movies_clean['vote_average'].max():.1f}")
print(f"Runtime range: {movies_clean['runtime'].min():.0f} to {movies_clean['runtime'].max():.0f} minutes")

=== DATA CLEANING STEP 2: Remove Duplicates & Filter Quality Data ===
Original dataset size: 4803 movies
After removing duplicates: 4800 movies

Applying quality filters...
Before quality filter: 4800 movies
After quality filter: 3022 movies

Final dataset characteristics:
Year range: 1980.0 to 2016.0
Vote count range: 100 to 13752
Vote average range: 2.9 to 8.5
Runtime range: 63 to 242 minutes


In [5]:
print("=== DATA CLEANING STEP 3: Process Text Features (JSON Columns) ===")

# Helper functions for processing JSON-like columns
def safe_eval(x):
    """Safely evaluate string representations of lists/dicts"""
    try:
        return ast.literal_eval(x) if pd.notna(x) and x != '[]' else []
    except (ValueError, SyntaxError):
        return []

def extract_names(data_list, key='name', limit=None):
    """Extract names from list of dictionaries"""
    if isinstance(data_list, list):
        names = [item.get(key, '') for item in data_list if isinstance(item, dict) and item.get(key)]
        return names[:limit] if limit else names
    return []

# Process genres
print("Processing genres...")
movies_clean['genres_list'] = movies_clean['genres'].apply(safe_eval)
movies_clean['genres_names'] = movies_clean['genres_list'].apply(lambda x: extract_names(x))
movies_clean['genres_str'] = movies_clean['genres_names'].apply(lambda x: '|'.join(x) if x else '')

# Process keywords
print("Processing keywords...")
movies_clean['keywords_list'] = movies_clean['keywords'].apply(safe_eval)
movies_clean['keywords_names'] = movies_clean['keywords_list'].apply(lambda x: extract_names(x, limit=10))

# Process production companies
print("Processing production companies...")
movies_clean['production_companies_list'] = movies_clean['production_companies'].apply(safe_eval)
movies_clean['production_companies_names'] = movies_clean['production_companies_list'].apply(lambda x: extract_names(x, limit=3))

# Merge with credits to get cast information
print("Processing cast information...")
movies_clean = movies_clean.merge(credits[['movie_id', 'cast']], left_on='id', right_on='movie_id', how='left')
movies_clean['cast_list'] = movies_clean['cast'].apply(safe_eval)
movies_clean['cast_names'] = movies_clean['cast_list'].apply(lambda x: extract_names(x, limit=5))

print("Text feature processing completed!")

# Show examples of processed features
print(f"\nExample processed features:")
sample_idx = 0
print(f"Movie: {movies_clean.iloc[sample_idx]['title']}")
print(f"Genres: {movies_clean.iloc[sample_idx]['genres_names']}")
print(f"Keywords: {movies_clean.iloc[sample_idx]['keywords_names'][:5]}")  # Show first 5
print(f"Cast: {movies_clean.iloc[sample_idx]['cast_names']}")
print(f"Production Companies: {movies_clean.iloc[sample_idx]['production_companies_names']}")

=== DATA CLEANING STEP 3: Process Text Features (JSON Columns) ===
Processing genres...
Processing keywords...
Processing production companies...
Processing cast information...
Text feature processing completed!

Example processed features:
Movie: Avatar
Genres: ['Action', 'Adventure', 'Fantasy', 'Science Fiction']
Keywords: ['culture clash', 'future', 'space war', 'space colony', 'society']
Cast: ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Michelle Rodriguez']
Production Companies: ['Ingenious Film Partners', 'Twentieth Century Fox Film Corporation', 'Dune Entertainment']


In [6]:
print("=== DATA CLEANING STEP 4: Create Final Cleaned Dataset ===")

# Select relevant columns for movie recommender system
columns_for_recommender = [
    'id', 'title', 'overview', 'tagline',
    'genres_str', 'genres_names', 'keywords_names',
    'release_date', 'release_year', 'budget', 'revenue', 'runtime',
    'popularity', 'vote_average', 'vote_count',
    'production_companies_names', 'cast_names',
    'original_language', 'status'
]

# Keep only columns that exist in our dataset
existing_cols = [col for col in columns_for_recommender if col in movies_clean.columns]
movies_final = movies_clean[existing_cols].copy()

print(f"Selected {len(existing_cols)} columns for final dataset")
print(f"Columns: {existing_cols}")

# Handle missing values appropriately
print("\nHandling missing values...")

# Fill missing taglines
if 'tagline' in movies_final.columns:
    movies_final['tagline'] = movies_final['tagline'].fillna('')

# Fill missing runtime with median
if 'runtime' in movies_final.columns:
    movies_final['runtime'] = movies_final['runtime'].fillna(movies_final['runtime'].median())

# Fill missing budget and revenue with 0
movies_final['budget'] = movies_final['budget'].fillna(0)
movies_final['revenue'] = movies_final['revenue'].fillna(0)

print("Missing value handling completed!")

# Create combined features for content-based filtering
print("\nCreating combined features for content-based filtering...")

def create_combined_features(row):
    """Combine all text features into one string for content-based filtering"""
    features = []
    
    # Add genres
    if 'genres_names' in row and isinstance(row['genres_names'], list):
        features.extend(row['genres_names'])
    
    # Add top keywords
    if 'keywords_names' in row and isinstance(row['keywords_names'], list):
        features.extend(row['keywords_names'][:5])  # Top 5 keywords
    
    # Add top cast members
    if 'cast_names' in row and isinstance(row['cast_names'], list):
        features.extend(row['cast_names'][:3])  # Top 3 actors
    
    # Add top production companies
    if 'production_companies_names' in row and isinstance(row['production_companies_names'], list):
        features.extend(row['production_companies_names'][:2])  # Top 2 companies
    
    return ' '.join(features).lower()

movies_final['combined_features'] = movies_final.apply(create_combined_features, axis=1)

print("Combined features created successfully!")
print(f"\nExample combined features:")
print(f"Movie: {movies_final.iloc[0]['title']}")
print(f"Combined: {movies_final.iloc[0]['combined_features'][:100]}...")  # First 100 chars

=== DATA CLEANING STEP 4: Create Final Cleaned Dataset ===
Selected 19 columns for final dataset
Columns: ['id', 'title', 'overview', 'tagline', 'genres_str', 'genres_names', 'keywords_names', 'release_date', 'release_year', 'budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count', 'production_companies_names', 'cast_names', 'original_language', 'status']

Handling missing values...
Missing value handling completed!

Creating combined features for content-based filtering...
Combined features created successfully!

Example combined features:
Movie: Avatar
Combined: action adventure fantasy science fiction culture clash future space war space colony society sam wor...


In [7]:
print("=== FINAL DATASET SUMMARY & EXPORT ===")

# Save the cleaned dataset to data/dataset/ directory
movies_final.to_csv('data/dataset/movies_clean.csv', index=False)
print("Dataset saved as 'data/dataset/movies_clean.csv'")

# Display final summary statistics
print(f"\n📊 FINAL CLEANED DATASET SUMMARY:")
print(f"Total movies: {len(movies_final):,}")
print(f"Total columns: {len(movies_final.columns)}")
print(f"Memory usage: {movies_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Show dataset characteristics
print(f"\n🎬 Dataset Characteristics:")
print(f"Year range: {movies_final['release_year'].min():.0f} - {movies_final['release_year'].max():.0f}")
print(f"Vote count range: {movies_final['vote_count'].min():.0f} - {movies_final['vote_count'].max():.0f}")
print(f"Vote average range: {movies_final['vote_average'].min():.1f} - {movies_final['vote_average'].max():.1f}")
print(f"Runtime range: {movies_final['runtime'].min():.0f} - {movies_final['runtime'].max():.0f} minutes")

# Show genre distribution
if 'genres_names' in movies_final.columns:
    print(f"\n🎭 Top 10 Genres:")
    all_genres = []
    for genres_list in movies_final['genres_names']:
        if isinstance(genres_list, list):
            all_genres.extend(genres_list)
    
    genre_counts = pd.Series(all_genres).value_counts().head(10)
    for genre, count in genre_counts.items():
        print(f"  {genre}: {count:,} movies")

# Show language distribution
print(f"\n🌍 Top Languages:")
lang_counts = movies_final['original_language'].value_counts().head(5)
for lang, count in lang_counts.items():
    print(f"  {lang}: {count:,} movies")

print(f"\n✅ DATA CLEANING COMPLETED!")
print(f"✅ Your movie dataset is ready for building a recommender system!")
print(f"✅ You can now use this clean data for:")
print(f"   • Content-based filtering (using genres, cast, keywords)")
print(f"   • Collaborative filtering (using ratings)")
print(f"   • Hybrid recommender systems")

# Show final column list
print(f"\n📝 Final dataset columns:")
for i, col in enumerate(movies_final.columns, 1):
    print(f"  {i:2d}. {col}")

=== FINAL DATASET SUMMARY & EXPORT ===
Dataset saved as 'data/dataset/movies_clean.csv'

📊 FINAL CLEANED DATASET SUMMARY:
Total movies: 3,022
Total columns: 20
Memory usage: 4.00 MB

🎬 Dataset Characteristics:
Year range: 1980 - 2016
Vote count range: 100 - 13752
Vote average range: 2.9 - 8.5
Runtime range: 63 - 242 minutes

🎭 Top 10 Genres:
  Drama: 1,302 movies
  Comedy: 1,092 movies
  Thriller: 941 movies
  Action: 857 movies
  Adventure: 625 movies
  Crime: 515 movies
  Romance: 512 movies
  Science Fiction: 414 movies
  Horror: 368 movies
  Fantasy: 356 movies

🌍 Top Languages:
  en: 2,888 movies
  fr: 37 movies
  es: 19 movies
  ja: 10 movies
  zh: 10 movies

✅ DATA CLEANING COMPLETED!
✅ Your movie dataset is ready for building a recommender system!
✅ You can now use this clean data for:
   • Content-based filtering (using genres, cast, keywords)
   • Collaborative filtering (using ratings)
   • Hybrid recommender systems

📝 Final dataset columns:
   1. id
   2. title
   3. overv